###Mounting datalake gen 2 storage

In [2]:
tenantID = "1275dea2-b646-4021-8554-5a4100c6b5e0"

storageAccountName = "psdemostorageacc2"
fileSystemName = "psdemofs"

In [3]:

configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": dbutils.secrets.get(scope = "key_vault_secrets", key = "azure-ad-app-id"), # retrieving app id from the key vault
           "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope = "key_vault_secrets", key = "azure-ad-secret-key"), # retrieving app secret from the key vault
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://{}@{}.dfs.core.windows.net/".format(fileSystemName, storageAccountName),
  mount_point = "/mnt/datalake",
  extra_configs = configs)


Out[4]: True

###Mounting blob storage

In [5]:
container_name = "fhvbases"
storage_account_name = "psdemoblob2"

In [6]:
dbutils.fs.mount(
  source = "wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/",
  mount_point = "/mnt/storage",
  extra_configs = {
      "fs.azure.account.key." + storage_account_name + ".blob.core.windows.net" : dbutils.secrets.get(scope = "key_vault_secrets", key = "ps-demo-blob-access-key")
  }
)

Out[16]: True

###Making sure the mount is successfull by accessing the files in bith bolb storage and datalake gen 2 storage

In [8]:
dbutils.fs.ls('/mnt/storage/')

Out[1]: [FileInfo(path='dbfs:/mnt/storage/FhvBases.json', name='FhvBases.json', size=464836)]

In [9]:
dbutils.fs.head('/mnt/storage/FhvBases.json')

[Truncated to first 65536 bytes]
Out[2]: '[\r\n {\r\n "License Number":"B02865"\r\n ,"Entity Name":"VIER-NY,LLC"\r\n ,"Telephone Number":6466657536\r\n ,"SHL Endorsed":"No"\r\n , "Address" : \r\n { \r\n "Building":"636"\r\n ,"Street":"WEST 28 STREET"\r\n ,"City":"NEW YORK"\r\n ,"State":"NY"\r\n ,"Postcode":10001\r\n }\r\n , "GeoLocation" : \r\n { "Latitude":40.75273\r\n ,"Longitude":-74.006408\r\n ,"Location":"(40.75273, -74.006408)"\r\n }\r\n ,"Type of Base":"BLACK CAR BASE" \r\n ,"Date":"08/15/2019"\r\n ,"Time":"18:03:31" \r\n }\r\n,\r\n {\r\n "License Number":"B02634"\r\n ,"Entity Name":"VETERANS RADIO DISPATCHER CORP."\r\n ,"Telephone Number":7183647878\r\n ,"SHL Endorsed":"No"\r\n , "Address" : \r\n { \r\n "Building":"131"\r\n ,"Street":"KINGSBRIDGE ROAD"\r\n ,"City":"BRONX"\r\n ,"State":"NY"\r\n ,"Postcode":10468\r\n }\r\n , "GeoLocation" : \r\n { "Latitude":40.86927\r\n ,"Longitude":-73.90281\r\n ,"Location":"(40.86927, -73.90281)"\r\n }\r\n ,"Type of Base":"LIVERY BASE" \r\n ,"Date":"08/15/2019"\r\n ,"Time":"18:03:31" \r\n }\r\n,\r\n {\r\n "License Number":"B80094"\r\n ,"Entity Name":"ALPHA VAN LINE"\r\n ,"Telephone Number":5162850750\r\n ,"SHL Endorsed":"No"\r\n , "Address" : \r\n { \r\n "Building":"115-54"\r\n ,"Street":"238 STREET"\r\n ,"City":"ELMONT"\r\n ,"State":"NY"\r\n ,"Postcode":11003\r\n }\r\n , "GeoLocation" : \r\n { "Latitude":40.693473\r\n ,"Longitude":-73.724446\r\n ,"Location":"(40.693473, -73.724446)"\r\n }\r\n ,"Type of Base":"COMMUTER VAN AUTHORITY BASE" \r\n ,"Date":"08/15/2019"\r\n ,"Time":"18:03:31" \r\n }\r\n,\r\n {\r\n "License Number":"B02677"\r\n ,"Entity Name":"A.T.B. CAR AND LIMOUSINE SERVICE, INC."\r\n ,"Telephone Number":7184854444\r\n ,"SHL Endorsed":"No"\r\n , "Address" : \r\n { \r\n "Building":"866"\r\n ,"Street":"NEW LOTS AVENUE"\r\n ,"City":"BROOKLYN"\r\n ,"State":"NY"\r\n ,"Postcode":11208\r\n }\r\n , "GeoLocation" : \r\n { "Latitude":40.667838\r\n ,"Longitude":-73.8788\r\n ,"Location":"(40.667838, -73.8788)"\r\n }\r\n ,"Type of Base":"LIVERY BASE" \r\n ,"Date":"08/15/2019"\r\n ,"Time":"18:03:31" \r\n }\r\n,\r\n {\r\n "License Number":"B02152"\r\n ,"Entity Name":"KYOEI LIMOUSINE, INC."\r\n ,"Telephone Number":7183263258\r\n ,"SHL Endorsed":"No"\r\n , "Address" : \r\n { \r\n "Building":"57-48"\r\n ,"Street":"MASPETH AVENUE"\r\n ,"City":"MASPETH"\r\n ,"State":"NY"\r\n ,"Postcode":11378\r\n }\r\n , "GeoLocation" : \r\n { "Latitude":40.722961\r\n ,"Longitude":-73.91031\r\n ,"Location":"(40.722961, -73.91031)"\r\n }\r\n ,"Type of Base":"LUXURY/LIMOUSINE" \r\n ,"Date":"08/15/2019"\r\n ,"Time":"18:03:31" \r\n }\r\n,\r\n {\r\n "License Number":"B02844"\r\n ,"Entity Name":"ENDOR CAR & DRIVER,LLC."\r\n ,"Telephone Number":4154758459\r\n ,"SHL Endorsed":"No"\r\n , "Address" : \r\n { \r\n "Building":"31-00"\r\n ,"Street":"47 AVENUE SUITE # 4123A"\r\n ,"City":"LIC"\r\n ,"State":"NY"\r\n ,"Postcode":11101\r\n }\r\n , "GeoLocation" : \r\n { "Latitude":40.742082\r\n ,"Longitude":-73.93552\r\n ,"Location":"(40.742082, -73.93552)"\r\n }\r\n ,"Type of Base":"BLACK CAR BASE" \r\n ,"Date":"08/15/2019"\r\n ,"Time":"18:03:31" \r\n }\r\n,\r\n {\r\n "License Number":"B02841"\r\n ,"Entity Name":"SKYWAY EXECUTIVE SERVICE, INC"\r\n ,"Telephone Number":7183595959\r\n ,"SHL Endorsed":"No"\r\n , "Address" : \r\n { \r\n "Building":"68-20A"\r\n ,"Street":"FRESH MEADOW LANE"\r\n ,"City":"FRESH MEADOWS"\r\n ,"State":"NY"\r\n ,"Postcode":11365\r\n }\r\n , "GeoLocation" : \r\n { "Latitude":40.733337\r\n ,"Longitude":-73.794706\r\n ,"Location":"(40.733337, -73.794706)"\r\n }\r\n ,"Type of Base":"BLACK CAR BASE" \r\n ,"Date":"08/15/2019"\r\n ,"Time":"18:03:31" \r\n }\r\n,\r\n {\r\n "License Number":"B00472"\r\n ,"Entity Name":"FARRELL\'S LEASING CO."\r\n ,"Telephone Number":2128616300\r\n ,"SHL Endorsed":"No"\r\n , "Address" : \r\n { \r\n "Building":"22-11"\r\n ,"Street":"38 AVENUE"\r\n ,"City":"LIC"\r\n ,"State":"NY"\r\n ,"Postcode":11101\r\n }\r\n , "GeoLocation" : \r\n { "Latitude":40.757077\r\n ,"Longitude":-73.937504

In [10]:
dbutils.fs.head('/mnt/datalake/PaymentTypes.json')

Out[6]: '{"PaymentTypeID":1,"PaymentType":"Credit Card"}\r\n{"PaymentTypeID":2,"PaymentType":"Cash"}\r\n{"PaymentTypeID":3,"PaymentType":"No Charge"}\r\n{"PaymentTypeID":4,"PaymentType":"Dispute"}\r\n{"PaymentTypeID":5,"PaymentType":"Unknown"}\r\n{"PaymentTypeID":6,"PaymentType":"Voided Trip"}\r\n'

###Creating the database we will be using for the project

In [12]:
%sql

create database if not exists taxi_service_warehouse;

##The following commands are not relevant to this file. Just to know a few commands and how they work

In [14]:
%fs ls /mnt/datalake/

path,name,size
dbfs:/mnt/datalake/GreenTaxiTripData_201812.csv,GreenTaxiTripData_201812.csv,62056490
dbfs:/mnt/datalake/PaymentTypes.json,PaymentTypes.json,277
dbfs:/mnt/datalake/RateCodes.csv,RateCodes.csv,139
dbfs:/mnt/datalake/RateCodes.json,RateCodes.json,252
dbfs:/mnt/datalake/TaxiZones.csv,TaxiZones.csv,12322
dbfs:/mnt/datalake/yellow_tripdata_2018-12.csv,yellow_tripdata_2018-12.csv,721522221


In [15]:
%sh ls -l /mnt/

total 8
drwxr-xr-x 2 root root 4096 Feb 2 14:22 driver-daemon
drwxr-xr-x 2 nobody nogroup 4096 Feb 2 14:22 readonly

###Old account details and useful links (DO NOT RUN THE following cell)

In [17]:
# Mounting datalake gen 2 account using appid and service principal
'''
https://docs.databricks.com/data/data-sources/azure/azure-datalake-gen2.html#mount-an-azure-data-lake-storage-gen2-account-using-a-service-principal-and-oauth-20
'''

appID = "1e6e56a8-********"
secret = "._d57tzy*******"
tenantID = "600b1b82-******"

storageAccountName = "psdemostoacc"
fileSystemName = "psdemofs"

configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": appID,
           "fs.azure.account.oauth2.client.secret": secret,
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://{}@{}.dfs.core.windows.net/".format(fileSystemName, storageAccountName),
  mount_point = "/mnt/datalake",
  extra_configs = configs)

'''
You can also access storage without mounting using

appID = "1e6e56a8-********"
secret = "._d57tzy*******"
tenantID = "600b1b82-******"

storageAccountName = "psdemostoacc"
fileSystemName = "psdemofs"

spark.conf.set("fs.azure.account.auth.type." + storageAccountName + ".dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type." + storageAccountName + ".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id." + storageAccountName + ".dfs.core.windows.net", "" + appID + "")
spark.conf.set("fs.azure.account.oauth2.client.secret." + storageAccountName + ".dfs.core.windows.net", "" + secret + "")
spark.conf.set("fs.azure.account.oauth2.client.endpoint." + storageAccountName + ".dfs.core.windows.net", "https://login.microsoftonline.com/" + tenantID + "/oauth2/token")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")
'''


  
  
  ----------------  ----------------  ----------------  ----------------  ----------------  ----------------  ----------------
  
  
# Mounting Blob container using access key
'''
https://docs.databricks.com/data/data-sources/azure/azure-storage.html#mount-azure-blob-storage
https://docs.databricks.com/_static/notebooks/data-sources/mount-azure-blob-storage.html
'''
container_name = "fhvbases"
storage_account_name = "psdemoblob"
storage_account_access_key = "kfnur***********"

dbutils.fs.mount(
  source = "wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/",
  mount_point = "/mnt/storage",
  extra_configs = {
      "fs.azure.account.key." + storage_account_name + ".blob.core.windows.net" : storage_account_access_key
  }
)